In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image 
import time
from skimage import io
from skimage.morphology import convex_hull_image
from skimage.measure import label, regionprops
from skimage.color import rgb2gray
from PIL import Image 
import coremltools 

In [ ]:
cap = cv2.VideoCapture(1)
time.sleep(0)

In [ ]:
model = coremltools.models.MLModel('ColourCheck.mlmodel')

In [ ]:
while (serial):
    ret, frame = cap.read() # ret is boolean for pic avalble or not
    frame=frame[...,::-1] # cv2 BRG to RGB
    grayframe=rgb2gray(frame)
    binary=grayframe>200
    labelframe=label(binary)
    regiongrframe=regionprops(labelframe)
    
    labelreal=-1
    yreal=0
    minr, minc, maxr, maxc=0,0,0,0
    for j in range(len(regiongrframe)):
        area=regiongrframe[j].area
        y0,x0=regiongrframe[j].centroid
        if area<5000:
            pass
        elif(y0>yreal):
            labelreal= regiongrframe[j].label
            yreal=y0
            minr, minc, maxr, maxc = region.bbox
    
    binary=(labelframe==labelreal)
    frame=frame*binary
    binary=binary[minr:maxr+1,minc,maxc+1]
    frame=frame[minr:maxr+1,minc,maxc+1,:]
    
    R=frame[:,:,0]
    G=frame[:,:,1]
    B=frame[:,:,2]
    meanofrgb=(R+G+B)/1.65
    
    biforblen=binary&(R>G*1.5)
    if (np.sum(biforblen)*100/np.sum(convex_hull_image(biforblen))<70):
        #return false
        continue
    
    frame[(R+G+B)==0]=255
    img=Image.fromarray(frame)
    
    if(model.predict({'image': img})['classLabel']=='true'):
        #return true
    else:
        #return false

In [ ]:
cap.release()
cv2.destroyAllWindows()